# This is just a first tests. Don't judge me

### Importing and preprocessing of data

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# read in csv data
matches = pd.read_csv("AI Match Results 150years.csv")
matches.head()

In [ ]:
matches.shape

In [ ]:
matches.dtypes

In [ ]:
import numpy as np

#convert date into usable format
matches["date"] = pd.to_datetime(matches["date"])

# convert object columns into integers
matches["home_team_code"] = matches["home_team"].astype("category").cat.codes
matches["away_team_code"] = matches["away_team"].astype("category").cat.codes
matches["tournament_code"] = matches["tournament"].astype("category").cat.codes

# convert result into boolean statements
matches["home_win"] = (matches["home_score"] > matches["away_score"])
matches["away_win"] = (matches["home_score"] < matches["away_score"])
matches["draw"] = (matches["home_score"] == matches["away_score"])

matches["result"] = "Home_Win"
matches.loc[matches["home_score"] < matches["away_score"], "result"] = "Away_Win"
matches.loc[matches["home_score"] == matches["away_score"], "result"] = "Draw"
matches["result_code"] = matches["result"].astype("category").cat.codes


#drop old data before 1960
#matches  = matches.drop(matches[matches['date'] < '1960-01-01'].index)

# dunno why datetime does not work for training. Therefore creating a new attribute with just rising numbers 
matches["date_delta"] = (matches['date'] - matches['date'].min())  / np.timedelta64(1,'D')

matches.head(100)

### Spliting the data for training

In [ ]:
# split train and test data: use 80% for training and the remaining for testing
train = matches.sample(frac = 0.8)
test  = matches.drop(train.index)

train.shape

In [ ]:
test.shape

# RandomForestClassifier

In [ ]:
# fitting
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

predictors = ["date_delta", "home_team_code", "away_team_code", "tournament_code", "neutral"]
targets = "home_win"

rf.fit(train[predictors], train[targets])



In [ ]:
# make predictions
predictions = rf.predict(test[predictors])

# accuracy
acc_score = accuracy_score(test[targets], predictions)
acc_score


In [ ]:
#visualize the predcitions a bit
combined = pd.DataFrame(dict(actual=test[targets], predicted=predictions))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

# MLPClassifier

In [ ]:
# fitting
from sklearn.neural_network import MLPClassifier

predictors = ["home_team_code", "away_team_code"]
targets = "result_code"

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(30, 5), random_state=2)

clf.fit(train[predictors], train[targets])

In [ ]:
# make predictions
predictions = clf.predict(test[predictors])

# accuracy
acc_score = accuracy_score(test[targets], predictions)
acc_score

In [ ]:
#visualize the predcitions a bit
combined = pd.DataFrame(dict(actual=test[targets], predicted=predictions))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

# That doesn't work at all. Time to improve the Data

How about calculating the strength of each team using ELO
https://en.wikipedia.org/wiki/World_Football_Elo_Ratings

In [ ]:
# create dict for tournament type

# to output all torunament types print the following line
#matches["tournament"].unique()

# set according to http://www.eloratings.net/about
K = {
    'Friendly': 20,
    'British Championship': 30,
    'Copa Lipton': 30,
    'Copa Newton': 30,
    'Copa Premio Honor Argentino': 30,
    'Copa Premio Honor Uruguayo': 30,
    'Copa Roca': 30,
    'Copa América': 30,
    'Copa Chevallier Boutell': 30,
    'Nordic Championship': 30,
    'International Cup': 50,
    'Baltic Cup': 30,
    'Balkan Cup': 30,
    'FIFA World Cup': 60,
    'Copa Rio Branco': 30,
    'FIFA World Cup qualification': 50,
    'Bolivarian Games': 30,
    'CCCF Championship': 30,
    'NAFU Championship': 30,
    'Copa Oswaldo Cruz': 30,
    'Pan American Championship': 30,
    'Copa del Pacífico': 30,
    "Copa Bernardo O'Higgins": 30,
    'AFC Asian Cup qualification': 30,
    'Atlantic Cup': 30,
    'AFC Asian Cup': 30,
    'African Cup of Nations': 30,
    'Copa Paz del Chaco': 30,
    'Merdeka Tournament': 30,
    'UEFA Euro qualification': 40,
    'UEFA Euro': 50,
    'Windward Islands Tournament': 30,
    'African Cup of Nations qualification': 30,
    'Vietnam Independence Cup': 30,
    'Copa Carlos Dittborn': 30,
    'Phillip Seaga Cup': 30,
    'CONCACAF Championship': 30,
    'Copa Juan Pinto Durán': 30,
    'Arab Cup': 30,
    'South Pacific Games': 30,
    'CONCACAF Championship qualification': 30,
    'Copa Artigas': 30,
    'GaNEFo': 30,
    "King's Cup": 30,
    'Gulf Cup': 30,
    'Indonesia Tournament': 30,
    'Korea Cup': 30,
    'Palestine Cup': 30,
    'Brazil Independence Cup': 30,
    'Copa Ramón Castilla': 30,
    'Oceania Nations Cup': 30,
    'CECAFA Cup': 30,
    'Kuneitra Cup': 30,
    'Copa Félix Bogado': 30,
    'Real Madrid 75th Anniversary Cup': 30,
    'Beijing International Friendship Tournament': 30,
    'Kirin Cup': 30,
    'CFU Caribbean Cup qualification': 30,
    'CFU Caribbean Cup': 30,
    'Amílcar Cabral Cup': 30,
    'FIFA 75th Anniversary Cup': 30,
    'Guangzhou International Friendship Tournament': 30,
    'Mundialito': 30,
    'West African Cup': 30,
    'Nehru Cup': 30,
    'Merlion Cup': 30,
    'Great Wall Cup': 30,
    'UDEAC Cup': 30,
    'Rous Cup': 30,
    'CONMEBOL–UEFA Cup of Champions': 30,
    'Lunar New Year Cup': 30,
    'Arab Cup qualification': 30,
    'Tournoi de France': 30,
    'Malta International Tournament': 30,
    'Matthews Cup': 30,
    'Tournament Burkina Faso': 30,
    'Island Games': 30,
    'Dynasty Cup': 30,
    'Dakar Tournament': 30,
    'UNCAF Cup': 30,
    'Gold Cup': 30,
    'USA Cup': 30,
    'Jordan International Tournament': 30,
    'Confederations Cup': 30,
    'United Arab Emirates Friendship Tournament': 30,
    'Oceania Nations Cup qualification': 30,
    'Simba Tournament': 30,
    'SAFF Cup': 30,
    'AFF Championship': 30,
    'King Hassan II Tournament': 30,
    'Cyprus International Tournament': 30,
    'Dunhill Cup': 30,
    'COSAFA Cup qualification': 30,
    'COSAFA Cup': 30,
    'Gold Cup qualification': 30,
    'SKN Football Festival': 30,
    'UNIFFAC Cup': 30,
    'WAFF Championship': 30,
    'Millennium Cup': 30,
    'Cup of Ancient Civilizations': 30,
    "Prime Minister's Cup": 30,
    'EAFF Championship': 30,
    'TIFOCO Tournament': 30,
    'Afro-Asian Games': 30,
    'AFC Challenge Cup': 30,
    'FIFI Wild Cup': 30,
    'ELF Cup': 30,
    'Viva World Cup': 30,
    'AFC Challenge Cup qualification': 30,
    'African Nations Championship': 30,
    'VFF Cup': 30,
    'Dragon Cup': 30,
    'ABCS Tournament': 30,
    'Nile Basin Tournament': 30,
    'Nations Cup': 30,
    'Copa Confraternidad': 30,
    'Pacific Games': 30,
    'Superclásico de las Américas': 30,
    'Kirin Challenge Cup': 30,
    'OSN Cup': 30,
    'CONIFA World Football Cup': 30,
    'CONIFA European Football Cup': 30,
    'Copa América qualification': 30,
    'World Unity Cup': 30,
    'Pacific Mini Games': 30,
    'Intercontinental Cup': 30,
    'AFF Championship qualification': 30,
    'UEFA Nations League': 40,
    'CONCACAF Nations League qualification': 30,
    'African Nations Championship qualification': 30,
    'Atlantic Heritage Cup': 30,
    'Inter Games Football Tournament': 30,
    'CONCACAF Nations League': 30,
    'Three Nations Cup': 30,
    'Mahinda Rajapaksa Cup': 30,
    'Navruz Cup': 30,
    'CONIFA Africa Football Cup': 30,
    'CONIFA South America Football Cup': 30,
    "MSG Prime Minister's Cup": 30
}

In [ ]:
def calcNewELO( R_old, R_opp, tournament_type, own_score, opp_score, is_home_team):
    # R_new = R_old + K * G * (W - W_e)
    #   R_new = The new team rating
    #   R_old = The old team rating
    #   K = Weight index regarding the tournament of the match
    #       World Cup, Olympic Games (1908–1980) 	60
    #       Continental championship and intercontinental tournaments 	50
    #       World Cup and Continental qualifiers and major tournaments 	40
    #       All other tournaments 	30
    #       Friendly matches 	30
    #   G = A number from the index of goal differences
    #   W = The result of the match
    #   W_e = The expected result
    G=1
    W=1

    # set G according to http://www.eloratings.net/about
    if (own_score >= (opp_score+4)):
        N = own_score-opp_score
        G = 1 + 3/4 + (N-3)/8
    elif (own_score > opp_score+3):
        G = 1.5

    # dr equals the difference in ratings plus 100 points for a team playing at home. 
    if (is_home_team):
        dr = R_old + 100 - R_opp
    else:
        dr = R_old - (R_opp +100)
    
    # We = 1 / ( (10^(-dr/400)) + 1)
    We = 1 / ( pow(10, (-dr/400)) + 1)

    if (own_score > opp_score):
        W = 1
    elif (own_score ==  opp_score):
        W = 0.5
    else:
        W = 0

    R_new = R_old + K[tournament_type] * G * (W - We)
    
    return R_new

In [ ]:
# Test it
Rn = calcNewELO(1000, 100, "UEFA Nations League", 5, 1, True )
print(Rn)

In [ ]:
# update matches data with ELO of each team
matches["ELO_home"] = 0
matches["ELO_away"] = 0

# make a list of all teams using "home_team" and "away_team"
uteams = (matches["home_team"].append( matches["away_team"] )).unique()
# initialie ELO with 1000
all_teams_ELO = dict.fromkeys(uteams, 1000)


for ind in matches.index:
    # set ELO from before the match
    ht = matches["home_team"][ind]
    at = matches["away_team"][ind]

    matches["ELO_home"][ind] = all_teams_ELO[ht]
    matches["ELO_away"][ind] = all_teams_ELO[at]

    # calc new ELO after match and update
    Rn_home = calcNewELO(matches["ELO_home"][ind], matches["ELO_away"][ind], matches["tournament"][ind], matches["home_score"][ind], matches["away_score"][ind], not matches["neutral"][ind])
    Rn_away = calcNewELO(matches["ELO_away"][ind], matches["ELO_home"][ind], matches["tournament"][ind], matches["away_score"][ind], matches["home_score"][ind], False)
    
    all_teams_ELO[ht] = Rn_home
    all_teams_ELO[at] = Rn_away
    
matches.head()

In [ ]:
all_teams_ELO

In [ ]:
matches.to_excel("with_ELO.xlsx")  

## Again MLPClassifier

In [ ]:
# split train and test data: use 80% for training and the remaining for testing
train = matches.sample(frac = 0.8)
test  = matches.drop(train.index)


# fitting
predictors = ["home_team_code", "away_team_code", "ELO_home", "ELO_away"]
targets = "result"

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(200, 10, 5), random_state=2)

clf.fit(train[predictors], train[targets])

In [ ]:
# make predictions
predictions = clf.predict(test[predictors])

# accuracy
acc_score = accuracy_score(test[targets], predictions)
acc_score

In [ ]:
#visualize the predcitions a bit
combined = pd.DataFrame(dict(actual=test[targets], predicted=predictions))
pd.crosstab(index=combined["predicted"], columns=combined["actual"])

## Again RandomForestClassifier

In [ ]:
# fitting
rf = RandomForestClassifier(n_estimators=200, min_samples_split=10, random_state=1)

predictors = ["home_team_code", "away_team_code", "ELO_home", "ELO_away", "neutral"]
targets = "home_win"

rf.fit(train[predictors], train[targets])

# make predictions
predictions = rf.predict(test[predictors])

# accuracy
acc_score = accuracy_score(test[targets], predictions)
acc_score

In [ ]:
#visualize the predcitions a bit
combined = pd.DataFrame(dict(actual=test[targets], predicted=predictions))
pd.crosstab(index=combined["predicted"], columns=combined["actual"])